### Завантаження та підготовка датасетів

In [1]:
import pandas as pd

dataset_train = pd.read_csv('dataset.csv')
dataset_train['label'].value_counts()

tell       431
show       179
unknown    147
Name: label, dtype: int64

In [2]:
with open('show-validation.txt', 'r') as f:
    show_sents = f.readlines()
    
with open('tell-validation.txt', 'r') as f:
    tell_sents = f.readlines()
    
dataset_test = pd.DataFrame({'sentence': show_sents + tell_sents,
                             'label': ['show'] * len(show_sents) + ['tell'] * len(tell_sents)})

dataset_test['label'].value_counts()

show    39
tell    26
Name: label, dtype: int64

In [3]:
import spacy
nlp = spacy.load("en_core_web_md")

In [4]:
dataset_train['sentence'] = dataset_train['sentence'].apply(nlp)
dataset_test['sentence'] = dataset_test['sentence'].apply(nlp)

### Роблю з тренувального датасету два датасети: перший буде без речень, проанотованих як 'unknown', а в другому - 'uknown' речення  будуть мати мітку 'show'.

In [5]:
no_unk = dataset_train[dataset_train['label'] != 'unknown']
no_unk['label'].value_counts()

tell    431
show    179
Name: label, dtype: int64

In [6]:
unk_as_show = dataset_train.replace({'label': {'unknown': 'show'}})
unk_as_show['label'].value_counts()

tell    431
show    326
Name: label, dtype: int64

### Перша версія класифікатора. Використовуватиму датасет без 'unknown' речень і просту токенізацію для BoW.

In [7]:
def tokenize(model):
    return [tok.text for tok in model]

def lemmatize(model):
    return [tok.lemma_ for tok in model]

In [8]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline
from sklearn.metrics import classification_report

def make_classifier():
    return Pipeline([('vect', CountVectorizer(lowercase=False, token_pattern=None)),
                     ('nb', MultinomialNB()),
                    ])

clf = make_classifier()
clf.set_params(vect__tokenizer=tokenize)

clf.fit(no_unk['sentence'], no_unk['label'])
print(classification_report(dataset_test['label'], clf.predict(dataset_test['sentence'])))

              precision    recall  f1-score   support

        show       1.00      0.26      0.41        39
        tell       0.47      1.00      0.64        26

    accuracy                           0.55        65
   macro avg       0.74      0.63      0.53        65
weighted avg       0.79      0.55      0.50        65



### Підбираю гіперпараметри для класифікатору

In [9]:
from sklearn.model_selection import GridSearchCV

parameter_grid = [{'vect__ngram_range': [(1, 1), (1, 2), (1, 3), (2, 2), (2, 3), (3, 3)],
                   'vect__tokenizer': [tokenize, lemmatize],
                   'nb__alpha': [1e-10, 0.001, 0.01, 0.05, 0.1, 0.3, 0.5, 1],
                   'nb__fit_prior': [True, False],
                  }]

gs_clf = GridSearchCV(make_classifier(), parameter_grid, scoring='f1_macro')
gs_clf.fit(no_unk['sentence'], no_unk['label'])
gs_clf.best_params_

{'nb__alpha': 0.3,
 'nb__fit_prior': False,
 'vect__ngram_range': (1, 1),
 'vect__tokenizer': <function __main__.tokenize(model)>}

### Результат трохи покращився

In [10]:
print(classification_report(dataset_test['label'], gs_clf.predict(dataset_test['sentence'])))

              precision    recall  f1-score   support

        show       0.75      0.38      0.51        39
        tell       0.47      0.81      0.59        26

    accuracy                           0.55        65
   macro avg       0.61      0.60      0.55        65
weighted avg       0.64      0.55      0.54        65



### Треную класифікатор на датасеті, де 'unknown'-и промарковані як 'show' і підбираю гіперпараметри для нього.

In [11]:
clf = make_classifier()
clf.set_params(vect__tokenizer=tokenize)

clf.fit(unk_as_show['sentence'], unk_as_show['label'])
print(classification_report(dataset_test['label'], clf.predict(dataset_test['sentence'])))

              precision    recall  f1-score   support

        show       0.82      0.46      0.59        39
        tell       0.51      0.85      0.64        26

    accuracy                           0.62        65
   macro avg       0.66      0.65      0.61        65
weighted avg       0.70      0.62      0.61        65



In [12]:
gs_clf = GridSearchCV(make_classifier(), parameter_grid, scoring='f1_macro')
gs_clf.fit(unk_as_show['sentence'], unk_as_show['label'])
gs_clf.best_params_

{'nb__alpha': 0.3,
 'nb__fit_prior': True,
 'vect__ngram_range': (1, 2),
 'vect__tokenizer': <function __main__.lemmatize(model)>}

### В порівнянні з попередньою версією, результат покращився

In [13]:
print(classification_report(dataset_test['label'], gs_clf.predict(dataset_test['sentence'])))

              precision    recall  f1-score   support

        show       0.81      0.54      0.65        39
        tell       0.54      0.81      0.65        26

    accuracy                           0.65        65
   macro avg       0.67      0.67      0.65        65
weighted avg       0.70      0.65      0.65        65



### Спостереження і висновки:
* розмір валідаційної вибірки досить малий, потрібно буде її розширити:
  * планую додати до неї речення, які я вислав Каті на доанотування;
  * якщо з доанотовуванням не вийде, то потрібно буде поскрейпити ще даних з сайтів з рекомендаціями для письменників;
* у мене була підозра, що при анотуванні, я помилково відносив деякі 'show' речення до 'unknown' категорії. саме тому, я вирішив зробити експерименти з двома датасетами. в експерименті з датасетом, де для 'uknown'-и були промарковані як 'show' якість виявилась кращою. це свідчить про те, що варто додатково переглянути 'unknown' категорію і можливо попереносити деякі речення в 'show'.
* BoW + NB показали хороший результат, я очікував гіршого. це водночас плюс (тому що якість хороша :)) і мінус (тому що складніше буде проводити покращення при розробці розумного рішення).